In [2]:
'''
匯入套件
'''
# 操作 browser 的 API
from selenium import webdriver

# 處理逾時例外的工具
from selenium.common.exceptions import TimeoutException

# 面對動態網頁，等待某個元素出現的工具，通常與 exptected_conditions 搭配
from selenium.webdriver.support.ui import WebDriverWait

# 搭配 WebDriverWait 使用，對元素狀態的一種期待條件，若條件發生，則等待結束，往下一行執行
from selenium.webdriver.support import expected_conditions as EC

# 期待元素出現要透過什麼方式指定，通常與 EC、WebDriverWait 一起使用
from selenium.webdriver.common.by import By

# 強制等待 (執行期間休息一下)
from time import sleep

# 整理 json 使用的工具
import json

# 執行 command 的時候用的
import os

'''
Selenium with Python 中文翻譯文檔
參考網頁：https://selenium-python-zh.readthedocs.io/en/latest/index.html
selenium 啓動 Chrome 的進階配置參數
參考網址：https://stackoverflow.max-everyday.com/2019/12/selenium-chrome-options/
Mouse Hover Action in Selenium
參考網址：https://www.toolsqa.com/selenium-webdriver/mouse-hover-action/
'''
# 啟動瀏覽器工具的選項
options = webdriver.ChromeOptions()
# options.add_argument("--headless")                #不開啟實體瀏覽器背景執行
options.add_argument("--start-maximized")         #最大化視窗
options.add_argument("--incognito")               #開啟無痕模式
options.add_argument("--disable-popup-blocking ") #禁用彈出攔截

# 使用 Chrome 的 WebDriver (含 options)
driver = webdriver.Chrome( options = options )

# 螢幕最大化
#driver.maximize_window()

# 放置爬取的資料
listData = []

'''
以 function 名稱，作為爬蟲流程
'''

# 走訪頁面
def visit():
    driver.get('https://www.youtube.com/');

# 輸入關鍵字
def search():
    # 輸入名稱
    txtInput = driver.find_element(By.CSS_SELECTOR, "input#search")
    txtInput.send_keys("張學友")
    
    #按下送出
    btnInput = driver.find_element(By.CSS_SELECTOR, "button#search-icon-legacy")
    btnInput.click()
    
# 篩選 (選項)
def filterFunc():
    try:
        # 等待篩選元素出現
        WebDriverWait(driver, 10).until(
            EC.presence_of_element_located( 
                (By.CSS_SELECTOR, "yt-formatted-string#text.style-scope.ytd-toggle-button-renderer.style-text") 
            )
        )

        #按下篩選元素，使項目浮現
        driver.find_element(By.CSS_SELECTOR, "yt-formatted-string#text.style-scope.ytd-toggle-button-renderer.style-text").click()

        # 等待一下
        sleep(2)

        # 按下選擇的項目
        driver.find_elements(By.CSS_SELECTOR, "yt-formatted-string.style-scope.ytd-search-filter-renderer")[10].click()
        
    except TimeoutException:
        print("等待逾時，即將關閉瀏覽器…")
        sleep(3)
        driver.quit()
        
# 滾動頁面
def scroll():
    # 瀏覽器內部的高度
    innerHeightOfWindow = 0
    
    # 當前捲動的量(高度)
    totalOffset = 0
    
    # 在捲動到沒有元素動態產生前，持續捲動
    while totalOffset <= innerHeightOfWindow:
        # 每次移動高度
        totalOffset += 300;
        
        # 捲動的 js code
        js_scroll = '''(
            function (){{
                window.scrollTo({{
                    top:{}, 
                    behavior: 'smooth' 
                }});
            }})();'''.format(totalOffset)
        
        # 執行 js code
        driver.execute_script(js_scroll)
        
        # 強制等待
        sleep(1)
        
        # 透過執行 js 語法來取得捲動後的當前總高度
        innerHeightOfWindow = driver.execute_script('return window.document.documentElement.scrollHeight;');
        
        # 強制等待
        sleep(1)
        
        # 印出捲動距離
        print("innerHeightOfWindow: {}, totalOffset: {}".format(innerHeightOfWindow, totalOffset))
        
        # 為了實驗功能，捲動超過一定的距離，就結束程式
        if totalOffset >= 300:
            break

# 分析頁面元素資訊
def parse():
    # 取得主要元素的集合
    ytd_video_renderers = driver.find_elements(By.CSS_SELECTOR, 'ytd-video-renderer.style-scope.ytd-item-section-renderer')
    
    # 逐一檢視元素
    for element in ytd_video_renderers:
        # 印出分隔文字
        print("=" * 30)
        
        # 取得圖片連結
        img = element.find_element(By.CSS_SELECTOR, "ytd-thumbnail.style-scope.ytd-video-renderer img#img")
        imgSrc = img.get_attribute('src')
        print(imgSrc)
        
        # 取得資料名稱
        a = element.find_element(By.CSS_SELECTOR, "a#video-title")
        aTitle = a.text
        print(aTitle)
        
        # 取得 YouTube 連結
        aLink = a.get_attribute('href')
        print(aLink)
        
        # 放資料到 list 中
        listData.append({
            "imgSrc": imgSrc,
            "aTitle": aTitle,
            "aLink": aLink
        })

# 將 list 存成 json
def saveJson():
    fp = open("youtube.json", "w", encoding='utf-8')
    fp.write( json.dumps(listData, ensure_ascii=False) )
    fp.close()
    
# 關閉瀏覽器
def close():
    driver.quit()

# ??????
def download():
    # 開啟 json 檔案
    fp = open("youtube.json", "r", encoding='utf-8')
    
    #取得 json 字串
    strJson = fp.read()
    
    # 關閉檔案
    fp.close()
    
    # 將 json 轉成 list (裡面是 dict 集合)
    listResult = json.loads(strJson)
    
    # ??????
    for index, obj in enumerate(listResult):
        if index <= 2:
            os.system("youtube-dl.exe -f mp4 -i {} -o {}".format(obj["aLink"], "%(id)s.%(ext)s"))
    


# 主程式
if __name__ == '__main__':
    visit()
    search()
    filterFunc()
    scroll()
    parse()
    saveJson()
    close()
    download()

innerHeightOfWindow: 5113, totalOffset: 300
https://i.ytimg.com/vi/LCcwt20QUWU/hq720.jpg?sqp=-oaymwEZCNAFEJQDSFXyq4qpAwsIARUAAIhCGAFwAQ==&rs=AOn4CLASKU-VdqVHotC7PMuVLF_3rqI6Ug
哈林想生就生　張學友：為老人爭面子｜三立新聞網SETN.com
https://www.youtube.com/watch?v=LCcwt20QUWU
https://i.ytimg.com/vi/mxzQrpTrhRk/hq720.jpg?sqp=-oaymwEZCNAFEJQDSFXyq4qpAwsIARUAAIhCGAFwAQ==&rs=AOn4CLB_OlfPnp7H969QnVPu62upDWYRfw
张学友 - 浪子心聲 [From the Heart of a Loafer]
https://www.youtube.com/watch?v=mxzQrpTrhRk
https://i.ytimg.com/vi/XNddVxU3Fzc/hqdefault.jpg?sqp=-oaymwEZCOADEI4CSFXyq4qpAwsIARUAAIhCGAFwAQ==&rs=AOn4CLALyXXl6hxPgk0ZgK3ahY1jQ8cRFA
如果這都不算愛 - 張學友
https://www.youtube.com/watch?v=XNddVxU3Fzc
None
Jacky Cheung - 張學友 - 回家的路彷彿太遠 (Official Video)
https://www.youtube.com/watch?v=YZ4ZX6fFj8k
None
張學友又抓到逃犯! 演唱會3通緝犯遭逮│中視新聞 20180522
https://www.youtube.com/watch?v=f3fBjSZ7qjA
None
56歲張學友劈一字馬　坦言「這是做最好的演唱會」
https://www.youtube.com/watch?v=KBZRJKlkvzA
None
鏡週刊 娛樂即時》張學友超強應援團 梁朝偉劉嘉玲嗑鍋 曾志偉醉翻
https://www.youtube.com/watch?v=KcOl1XK2DqA
Non